<a href='https://www.darshan.ac.in/'> <img src='https://www.darshan.ac.in/Content/media/DU_Logo.svg' width="250" height="300"/></a>
<pre>
<center><b><h1>Data Mining</b></center>
<center><b><h1>Lab - 10</b></center> 
<center><h2>23010101226 - Drashti Rathod - 516</h2></center>
<pre>



# Implement Decision Tree(ID3) in python
Uses Information Gain to choose the best feature to split. 

Recursively builds the tree until stopping conditions are met.

1) Calculate Entropy for the dataset.<BR>
2) Calculate Information Gain for each feature. <BR>
3) Choose the feature with maximum Information Gain. <BR>
4) Split dataset into subsets for that feature. <BR>
5) Repeat recursively until: <BR>

All samples in a node have the same label.<BR>
No features are left.<BR>
No data is left.

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

##  import Pandas, Numpy

In [1]:
import pandas as pd
import numpy as np

##  Create Following Data

In [16]:
data = pd.DataFrame({
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
})

In [17]:
print(data)

     Outlook Temperature Humidity    Wind PlayTennis
0      Sunny         Hot     High    Weak         No
1      Sunny         Hot     High  Strong         No
2   Overcast         Hot     High    Weak        Yes
3       Rain        Mild     High    Weak        Yes
4       Rain        Cool   Normal    Weak        Yes
5       Rain        Cool   Normal  Strong         No
6   Overcast        Cool   Normal  Strong        Yes
7      Sunny        Mild     High    Weak         No
8      Sunny        Cool   Normal    Weak        Yes
9       Rain        Mild   Normal    Weak        Yes
10     Sunny        Mild   Normal  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
13      Rain        Mild     High  Strong         No


## Now Define Function to  Calculate Entropy

In [18]:
def entropy(y):
    # Unique class labels and their counts
    elements, counts = np.unique(y, return_counts=True)
    ent = 0
    for i in range(len(elements)):
        prob = counts[i] / np.sum(counts)
        ent += -prob * np.log2(prob)
    return ent
  

## Testing of Above Function - 
y = np.array(['Yes', 'No',   'Yes', 'Yes'])  
Function Call - > entropy(y))

output - 0.8112781244591328

In [19]:
y = np.array(['Yes', 'No', 'Yes', 'Yes'])
print(entropy(y))

0.8112781244591328


## Define function to Calculate Information Gain

In [20]:
def information_gain(data, split_attribute, target):
   # Entropy of the whole dataset (before split)
    total_entropy = entropy(data[target])
    
    # Values & counts of the splitting attribute
    vals, counts = np.unique(data[split_attribute], return_counts=True)
    
    # Weighted entropy after split
    weighted_entropy = 0
    for i in range(len(vals)):
        subset = data[data[split_attribute] == vals[i]]
        prob = counts[i] / np.sum(counts)
        weighted_entropy += prob * entropy(subset[target])
    
    # Information Gain
    info_gain = total_entropy - weighted_entropy
    return info_gain

## Testing of Above Function- 
data = pd.DataFrame({
    'Weather': ['Sunny', 'Sunny', 'Rain', 'Rain'],
    'Play':    ['Yes', 'No',   'Yes', 'Yes']
})

Function Call - > information_gain(data, 'Weather', 'Play')


Output - 0.31127812445913283

In [9]:
data = pd.DataFrame({
    'Weather': ['Sunny', 'Sunny', 'Rain', 'Rain'],
    'Play': ['Yes', 'No', 'Yes', 'Yes']
})

print(information_gain(data, 'Weather', 'Play'))

0.31127812445913283


## Implement ID3 Algo

In [10]:
def id3(data, features, target):
    # If all labels are same → return the label
    if len(np.unique(data[target])) == 1:
        return np.unique(data[target])[0]
    
    # If no features left → return majority label
    if len(features) == 0:
        return np.unique(data[target])[np.argmax(
            np.unique(data[target], return_counts=True)[1]
        )]
    
    # Choose best feature
    gains = [information_gain(data, f, target) for f in features]
    best_feature = features[np.argmax(gains)]
    
    # Create tree as dictionary
    tree = {best_feature: {}}
    
    # For each value of the best feature → branch
    for value in np.unique(data[best_feature]):
        subset = data[data[best_feature] == value]
        # Remove used feature
        new_features = [f for f in features if f != best_feature]
        subtree = id3(subset, new_features, target)
        tree[best_feature][value] = subtree
    
    return tree

## Use ID3

In [21]:
features = list(data.columns[:-1])
target = 'PlayTennis'

tree = id3(data, features, target)

## Print Tree

In [22]:
print(tree)

{'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


## Extra: Create Predict Function

In [24]:
def predict(tree, sample):
   # If tree is just a label (string), return it
    if not isinstance(tree, dict):
        return tree
    
    # Get the root feature (first key of the dictionary)
    root_feature = next(iter(tree))
    
    # Get the feature value from the sample
    feature_value = sample[root_feature]
    
    # If the value exists in the tree, go deeper
    if feature_value in tree[root_feature]:
        subtree = tree[root_feature][feature_value]
        return predict(subtree, sample)
    else:
        # Handle unseen feature values (fallback)
        return None

##  Extra: Predict for a sample
sample = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'}


Your Answer ?

In [26]:
sample = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'}

prediction = predict(tree, sample)
print("Prediction:", prediction)

Prediction: No
